# Minimal crowd simulation example

This guide explains how to run multiple simulations using the **CrowdMechanics** library and how to manage their outputs.

---

## 1. Export crowd configuration files

1. Open the [**Streamlit app**](https://lemons.streamlit.app/).
2. Go to the **Crowd** tab.
3. Click **"Initialize your own crowd"**.
4. In the sidebar at the bottom, click **"Export crowd as XML config files"**.
5. Download the XML files and save them in an appropriate location.

---

## 2. Set the working environnement

Create two folders in your project directory `static` and `dynamic`. Place the following files that you downloaded from the Streamlit app in the respective folders:
  - Move `Agents.xml`, `Geometry.xml`, and `Materials.xml` into the `static` folder.
  - Move `AgentDynamics.xml` into the `dynamic` folder.

Create a `Parameters.xml` file in your actual folder location. Use the following template (replace the paths with your actual folder locations):

```xml
<?xml version="1.0" encoding="utf-8"?>
<Parameters>
    <Directories Static="./static/" Dynamic="./dynamic/"/>
    <Times TimeStep="0.1" TimeStepMechanical="5e-6"/>
</Parameters>
```

**Directory structure example:**
```
.
├── Parameters.xml
├── static/
|   ├── Agents.xml
│   ├── Geometry.xml
│   └── Materials.xml
└── dynamic/
    └── AgentDynamics.xml
```

---

## 3. Build the C++ project

Navigate to the root of the `mechanical_layer` directory and build the project:

```bash
cmake -H. -Bbuild -DBUILD_SHARED_LIBS=ON
cmake --build build
```

---

## 4. Run a series of simulations

Execute the Python code below, modifying it as needed for your experiments.
Simulation results are automatically saved in the `outputXML/` directory.
Each output file is named as follows:
```
AgentDynamics output t=TIME_VALUE.xml
```
Here, `TIME_VALUE` represents the specific simulation time or identifier for that run.



In [1]:
import ctypes
from pathlib import Path
import numpy as np
from shutil import copyfile
import xml.etree.ElementTree as ET

# === Simulation Parameters ===
dt = 0.1  # Time step for the decisional layer (matches "TimeStep" in Parameters.xml)
Ndt = 100  # How many dt will be performed in total

# === Paths Setup ===
outputPath = Path("outputXML/")  # Directory to store output XML files
inputPath = Path("inputXML/")  # Directory to store input XML files
outputPath.mkdir(parents=True, exist_ok=True)  # Create directories if they don't exist
inputPath.mkdir(parents=True, exist_ok=True)

# === Loading the External Mechanics Library ===
# Adjust filename for OS (.so for Linux, .dylib for macOS)
Clibrary = ctypes.CDLL("../../src/mechanical_layer/build/libCrowdMechanics.dylib")

agentDynamicsFilename = "AgentDynamics.xml"

# Prepare the list of XML files that will be passed to the DLL/shared library
files = [
    b"Parameters.xml",
    b"Materials.xml",
    b"Geometry.xml",
    b"Agents.xml",
    agentDynamicsFilename.encode("ascii"),  # Convert filename to bytes (required by ctypes)
]
nFiles = len(files)  # Number of configuration files to be passed
filesInput = (ctypes.c_char_p * nFiles)()  # Create a ctypes array of string pointers
filesInput[:] = files  # Populate array with the XML file names

# === Main Simulation Loop ===
for t in range(Ndt):
    print("Looping the Crowd mechanics engine - t=%.1fs..." % (t * dt))

    # 1. Save the current AgentDynamics file as input for this step (can be used for analysis later)
    copyfile("dynamic/" + agentDynamicsFilename, str(inputPath) + rf"/AgentDynamics input t={t * dt:.1f}.xml")

    # 2. Call the external mechanics engine, passing in the list of required XML files
    Clibrary.CrowdMechanics(filesInput)

    # 3. Save the updated AgentDynamics output to results folder (can be used for analysis later)
    copyfile("dynamic/" + agentDynamicsFilename, str(outputPath) + rf"/AgentDynamics output t={(t + 1) * dt:.1f}.xml")

    # 4. If the simulation produced an AgentInteractions.xml file, save that as well (optional output)
    try:
        copyfile("dynamic/AgentInteractions.xml", str(outputPath) + rf"/AgentInteractions t={(t + 1) * dt:.1f}.xml")
    except FileNotFoundError:
        # If the AgentInteractions file does not exist, skip copying
        pass

    # === Decision/Controller Layer for Next Step ===
    # Read the output AgentDynamics XML as input for the next run.
    # This is where you (or another program) can set new forces/moments for each agent for the next simulation step.
    XMLtree = ET.parse("dynamic/" + agentDynamicsFilename)
    agentsTree = XMLtree.getroot()

    # -- Assign random forces/moments to each agent --
    for agent in agentsTree:
        # Create new <Dynamics> tag for the agent (as the output file doesn't have it)
        dynamicsItem = ET.SubElement(agent, "Dynamics")

        # Assign random force, and random moment
        dynamicsItem.attrib["Fp"] = f"{np.random.normal(loc=200, scale=200):.2f},{np.random.normal(loc=0, scale=50):.2f}"
        dynamicsItem.attrib["Mp"] = f"{np.random.normal(loc=0, scale=5):.2f}"

    # Write the modified XML back, to be used in the next iteration
    XMLtree.write("dynamic/" + agentDynamicsFilename)
    # ================================================

# After all simulation steps are complete, print a final message.
print(f"Loop terminated at t={Ndt * dt:.1f}s!")


Looping the Crowd mechanics engine - t=0.0s...
Looping the Crowd mechanics engine - t=0.1s...
Looping the Crowd mechanics engine - t=0.2s...
Looping the Crowd mechanics engine - t=0.3s...
Looping the Crowd mechanics engine - t=0.4s...
Looping the Crowd mechanics engine - t=0.5s...
Looping the Crowd mechanics engine - t=0.6s...
Looping the Crowd mechanics engine - t=0.7s...
Looping the Crowd mechanics engine - t=0.8s...
Looping the Crowd mechanics engine - t=0.9s...
Looping the Crowd mechanics engine - t=1.0s...
Looping the Crowd mechanics engine - t=1.1s...
Looping the Crowd mechanics engine - t=1.2s...
Looping the Crowd mechanics engine - t=1.3s...
Looping the Crowd mechanics engine - t=1.4s...
Looping the Crowd mechanics engine - t=1.5s...
Looping the Crowd mechanics engine - t=1.6s...
Looping the Crowd mechanics engine - t=1.7s...
Looping the Crowd mechanics engine - t=1.8s...
Looping the Crowd mechanics engine - t=1.9s...
Looping the Crowd mechanics engine - t=2.0s...
Looping the C

For a c++ example file and precise explanations, read the `Mechanical layer` tutorial.

---

## 5. Export to ChAOS

The output files can be easily converted into `.csv` files structured for direct import into [ChAOS](https://project.inria.fr/crowdscience/project/ocsr/chaos/). ChAOS uses these CSV files—containing time, x, and y coordinates for each agent—to visualize and animate agent trajectories for further analysis and video generation.

In [ ]:
from pathlib import Path
from configuration.backup import xml_to_Chaos

filenameCSV = "all_trajectories.csv"  # Name of the final CSV file we’ll generate
PathXML = Path("inputXML").resolve()  # Folder path where the XML files are located
PathCSV = Path("inputCSV").resolve()  # Folder path where CSV files will be saved
PathCSV.mkdir(parents=True, exist_ok=True)  # Create directories if it doesn't exist

# Convert XML files to a single summarized CSV
xml_to_Chaos.export_dict_to_CSV(PathCSV, PathXML)

# Convert CSV to ChAOS format
xml_to_Chaos.export_from_CSV_to_CHAOS(PathCSV, dt)

Processing file: /Volumes/desk_oscar/main/cours/phd_first_year/shape_project/LEMONS/tutorials/mechanical_layer/inputXML/AgentDynamics input t=0.0.xml
Processing file: /Volumes/desk_oscar/main/cours/phd_first_year/shape_project/LEMONS/tutorials/mechanical_layer/inputXML/AgentDynamics input t=0.1.xml
Processing file: /Volumes/desk_oscar/main/cours/phd_first_year/shape_project/LEMONS/tutorials/mechanical_layer/inputXML/AgentDynamics input t=0.2.xml
Processing file: /Volumes/desk_oscar/main/cours/phd_first_year/shape_project/LEMONS/tutorials/mechanical_layer/inputXML/AgentDynamics input t=0.3.xml
Processing file: /Volumes/desk_oscar/main/cours/phd_first_year/shape_project/LEMONS/tutorials/mechanical_layer/inputXML/AgentDynamics input t=0.4.xml
Processing file: /Volumes/desk_oscar/main/cours/phd_first_year/shape_project/LEMONS/tutorials/mechanical_layer/inputXML/AgentDynamics input t=0.5.xml
Processing file: /Volumes/desk_oscar/main/cours/phd_first_year/shape_project/LEMONS/tutorials/mechan

---

## 6. Export to a movie with FFMPEG

A plot of the scene can be generated from each output file under `PNG` format. 

In [4]:
import os
import matplotlib.pyplot as plt
import configuration.backup.dict_to_xml_and_reverse as fun_xml  # For converting XML to dictionary and vice versa
from configuration.models.crowd import create_agents_from_dynamic_static_geometry_parameters  # For creating agents based on XML data
from streamlit_app.plot import plot  # For plotting crowd data

# === Simulation Parameters ===
dt = 0.1  # Time step for the decisional layer (matches "TimeStep" in Parameters.xml)
Ndt = 100  # How many dt will be performed in total

# === Prepare the folders ===
# Define the paths to the folders you'll use
outputPath = Path("outputXML/")
staticPath = Path("./static")
plotsPath = Path("./plots")
plotsPath.mkdir(parents=True, exist_ok=True) # Create plots directory if it doesn't exist

# Remove any old '.png' files in the plots directory
for file in plotsPath.glob("*.png"):
    os.remove(file)

# === Load static XML files ===
# Read the Agents.xml file as a string and convert it to a dictionary
with open(staticPath / "Agents.xml", encoding="utf-8") as f:
    crowd_xml = f.read()
static_dict = fun_xml.static_xml_to_dict(crowd_xml)

# Read the Geometry.xml file as a string and convert it to a dictionary
with open(staticPath / "Geometry.xml", encoding="utf-8") as f:
    geometry_xml = f.read()
geometry_dict = fun_xml.geometry_xml_to_dict(geometry_xml)

# === Loop over time steps ===
for t in range(Ndt):
    current_time = (t + 1) * dt

    # Check if the dynamics file exists; if not, skip to the next time step
    dynamics_file = outputPath / f"AgentDynamics output t={current_time:.1f}.xml"
    if not dynamics_file.exists():
        print(f"Warning: {dynamics_file} not found, skipping.")
        continue

    # === Read and process the dynamics XML file ===
    # Read the current dynamics XML file as a string and convert it to a dictionary
    with open(dynamics_file, encoding="utf-8") as f:
        dynamic_xml = f.read()
    dynamic_dict = fun_xml.dynamic_xml_to_dict(dynamic_xml)

    # Create a crowd object using the configuration files data
    crowd = create_agents_from_dynamic_static_geometry_parameters(
        static_dict=static_dict,
        dynamic_dict=dynamic_dict,
        geometry_dict=geometry_dict,
    )

    # Plot and save the crowd as a PNG file
    plot.display_crowd2D(crowd)
    plt.savefig(plotsPath / rf"crowd2D_t={t:d}.png", dpi=300, format="png")
    plt.close()

Below is an example of the type of plot you can generate:
<tr>
  <td align="center" style="width:100%;">
    <img src="./plots/crowd2D_t=50.png" width="450" alt="Snapshot from a crowd simulation video." style="display:block; margin:auto;">
  </td>
</tr>

This image illustrates a snapshot from a crowd simulation at a specific time step. You can finally combine all the `PNG` images into a video using [FFmpeg](https://ffmpeg.org/). Be sure to update the `FFmpeg` path in the script to match your system. The resulting video will be saved in `.mov` format.

In [5]:
import subprocess
from pathlib import Path

ffmpeg = "/Users/oscardufour/ffmpeg_bin/ffmpeg"
movie_name = "example"
plotsPath = Path("./plots")
moviesPath = Path("./movies")
moviesPath.mkdir(parents=True, exist_ok=True)
framerate = int(1.0 / dt)

# 1. Create an MP4 movie from PNG images in the plots folder
cmd1 = f"{ffmpeg} -framerate {framerate} -i {plotsPath}/crowd2D_t=%d.png {moviesPath}/{movie_name}.mp4"
subprocess.Popen(cmd1.split(), stdout=subprocess.PIPE).communicate()

# 2. Convert the MP4 to MOV for compatibility
cmd2 = f"{ffmpeg} -i {moviesPath}/{movie_name}.mp4 -pix_fmt yuv420p {moviesPath}/{movie_name}.mov"
subprocess.Popen(cmd2.split(), stdout=subprocess.PIPE).communicate()

# 3. Remove the intermediate MP4 file
cmd3 = f"rm {moviesPath}/{movie_name}.mp4"
subprocess.Popen(cmd3.split(), stdout=subprocess.PIPE).communicate()


ffmpeg version 7.0.2-tessus  https://evermeet.cx/ffmpeg/  Copyright (c) 2000-2024 the FFmpeg developers
  built with Apple clang version 15.0.0 (clang-1500.3.9.4)
  configuration: --cc=/usr/bin/clang --prefix=/opt/ffmpeg --extra-version=tessus --enable-avisynth --enable-fontconfig --enable-gpl --enable-libaom --enable-libass --enable-libbluray --enable-libdav1d --enable-libfreetype --enable-libgsm --enable-libharfbuzz --enable-libmodplug --enable-libmp3lame --enable-libmysofa --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenh264 --enable-libopenjpeg --enable-libopus --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvmaf --enable-libvo-amrwbenc --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxavs --enable-libxml2 --enable-libxvid --enable-libzimg --enable-libzmq --enable-libzvbi --disable-videotoolbox -

(b'', None)